<h1 align=center><font size = 5>Using FOURSQUARE data in COVID 19 times</font></h1>

## Introduction

Today COVID 19 is a real preocupation, so the objective of this project will be to use FOURSQUARE information in order to evaluate how close or far are main neighborhoods in the New York area from health services, mainly hospitals and pharmacies. Clinics will be out of the scope (as these are mostly classified by FOURSQUARE as cosmetic), and the initial sample will be from neighborhoods in Manhattan.  

This project will use information provided by FOURSQUARE regarding venues with tags like 'hospital' (both, public and private) and 'pharmacy'.  After a initial review, tags like 'clinic' although related to health, were mostly associated with cosmetic and even veterinary, so will be out of this scope.  As the focus will be New York City, USA; and using Manhatan for the first sample of neighborhoods, other datasets (like geolocation json) will be the same used in previous labs of the module.  For the analysis, initially was intended to use DBSCAN Clustering as it will allow to identify outliers (neighborhoods that may be considered isolated from health services), but after selecting to work with Manhattan as a sample, K-means will be better for clustering.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore the Data</a>

2. <a href="#item2">Explore Neighborhoods in Manhattan</a>

3. <a href="#item3">Search for Pharmacies and Hospitals</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before getting the data and start exploring it, download all needed dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore the Data

In order to segment the neighborhoods and explore them, a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood will be used. 

The link to the dataset of NYU Spatial Data Repository (2014): https://geo.nyu.edu/catalog/nyu_2451_34572

(the json file used is the one stored in Skills Network Labs)

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

## 2. Explore Neighborhoods in Manhattan

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [9]:
# @Hidden_Cel

CLIENT_ID = '1FXUNIGZUFV15SZNEKAV0ITPMXSZR5JD4W5IWXQUTQ25G0G4' # your Foursquare ID
CLIENT_SECRET = 'GCTV5G3TM225N40LPF5WLVMPF1Y2WSVR1V5E1MHM2MOAXQVY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


## 3. Search for Pharmacies and Hospitals

In [10]:
search_query = 'Hospital' 
search_query1 = 'Pharmacy'
radius = 500
LIMIT = 100

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
        url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query1, radius, LIMIT)
                
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        results1 = requests.get(url1).json()["response"]['venues']
        
         
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['name'],
            v['location']['lat'],
            v['location']['lng'],
            v['location']['distance']) for v in results])
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['location']['distance']) for v in results1])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Distance']
    
    return(nearby_venues)

In [12]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                    )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [13]:
print(manhattan_venues.shape)
manhattan_venues.head()

(785, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance
0,Marble Hill,40.876551,-73.91066,NewYork-Presbyterian/The Allen Hospital,40.873327,-73.913051,411
1,Marble Hill,40.876551,-73.91066,NYP Allen Hospital Emergency Department,40.872961,-73.913016,446
2,Marble Hill,40.876551,-73.91066,NYPH Allen Hospital Rm 081,40.873852,-73.913189,368
3,Marble Hill,40.876551,-73.91066,NewYork-Presbyterian Och Spine Hospital,40.873348,-73.913017,408
4,Marble Hill,40.876551,-73.91066,Allen Hospital Breastfeeding Fair,40.873348,-73.913017,408


In [14]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance
Neighborhood,,,,,,
Battery Park City,2,2,2,2,2,2
Carnegie Hill,16,16,16,16,16,16
Central Harlem,23,23,23,23,23,23
Chelsea,20,20,20,20,20,20
Chinatown,52,52,52,52,52,52
Civic Center,33,33,33,33,33,33
Clinton,7,7,7,7,7,7
East Harlem,15,15,15,15,15,15
East Village,17,17,17,17,17,17


In [15]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue'].unique())))

There are 519 uniques categories.


In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]





manhattan_onehot.head()

,Neighborhood,103 Pharmacy,110 pharmacy,133rd Street Pharmacy,139 Pharmacy,145th Street Animal Hospital,181 Pharmacy,678 Pharmacy,79th Street Pharmacy,8A Neuroscience Critical Care - Mt. Sinai Roosevelt Hospital,8B Neuroscience Patient Unit - Mt. Sinai Roosevelt Hospital,90th Street Pharmacy,98K Hospitality,A.M. Pharmacy II,ABC Animal Hospital,ABC Pharmacy,ABC Pharmacy / AID Pharmacy,ABC Pharmacy ; First Pharmaceutical Corp.,ABC Pharmacy V Corp,AIM Hospitality Group,APKG Pharmacy,ASPCA Adoption Center,ASPCA Animal Hospital,Abacus Pharmacy Inc.,Abc Pharmacy,Abu Dhabi Hospitality,Aid Pharmacy,Alisons Pharmacy,All Health Pharmacy,Allen Hospital Breastfeeding Fair,Alrite Pharmacy Inc.,Ambulatory Surgery at Roosevelt Hospital,American Academy of Hospitality Sciences,Animal Hospital Of Chelsea,Animal Hospital at Bideawee,Animal hospital,Apollo Pharmacy,Apotheco Pharmacy Manhattan - Dermatology Pharmacy,Apthorp Pharmacy,Arthur J. Antenucci Medical Research Building (The Roosevelt Hospital),Au Bon Pain,Avenue A Pharmacy,Avenue C pharmacy,B&B Hospitality Offices,BR Guest Hospitality,Bac Ai,Battery Park Pharmacy,Battery Park Veterinary Hospital,Bayard L.C. Pharmacy,Bayard L.C. Pharmacy Corp,Bayard Pharmacy,Bellevue Hospital 16W,Bellevue Hospital Center Gift Shop,Bellevue Hospital Center Pediatric Emergency,Bellevue Hospital Center Speciality Surgery,"Bellevue Hospital Center, 2nd Floor",Bellevue Pharmacy,Best Care Pharmacy,Best Western Plus Hospitality House,Beth Israel Hospital Department Of Neurology,Beth Israel Hospital Smoke Spot,Bickson Hospitality Group,Bio Care Pharmacy,Blake pharmacy,Bliss Pharmacy,Bliss Pharmacy Inc. / Upper Madison Drugs (DBA),Bowen Pharmacy,Bowery Pharmacy Inc,Brisely's Pharmacy,Broadway Family Pharmacy,Broadway Pharmacy,Brooklyn Navy Yard Hospital,Buy Rite Pharmacy,CCU @ Lenox Hill Hospital,CO Bigelow Pharmacy,CVS Pharmacy,CVS pharmacy,CVS/pharmacy Specialty Services,Cafe 58 @ Roosevelt Hospital,Caligore Pharmacy,Canal Street Pharmacy Inc.,Canalberry Pharmacy,CarePlus CVS/pharmacy,Carnegie Hill Pharmacy,Cathedral Dog & Cat Veterinary Hospital,Center Care Pharmacy,Century Pharmacy,Century pharmacy,Charlton Pharmacy,Chelsea Animal Hospital,Chelsea Royal Care Pharmacy,Cherry's Pharmacy,Christopher Park Veterinary Hospital,Cici Pharmacy,City Drug & Surgical Pharmacy,City Drug Pharmacy,Clean Care Pharmacy,Columbus Avenue Pharmacy,Community Pharmacy,Confucius Mannings Pharmacy,Confucius Pharmacy,Cooper Square Veterinary Hospital,Cordette Pharmacy,Cross County Pharmacy,Crown Hospitality Group,Cvs Pharmacy,Cvs Pharmacy In Target,Daliza Pharmacy,David’s Pharmacy,Denihan Hospitality Group,Department of Surgery-Bellevue Hospital,Destinations Link Hospitality,Dette Pharmacy,Dichter Pharmacy,Downtown Pharmacy,Drug Loft Pharmacy,Duane Reade,Duane Reade Pharmacy,E Care Pharmacy,ESquared Hospitality,EZ Care Pharmacy,East Village Prescription Center,Eastside Animal Hospital,Elliott Pharmacy,EmblemHealth Pharmacy,Empire Pharmacy,Erno Laszlo Counter at Apthorp Pharmacy,Esquire Pharmacy,Eve Pharmacy & Optical,"Fadi F. Attiyeh, MD",Falk Pharmacy,Family Choice Pharmacy Corp.,Family Plus Pharmacy Corp,Fancy Pharmacy,Feins Ethical Pharmacy,First Ave Veterinary Hospital and Dental Clinic,First Avenue Pharmacy,Forever Health Pharmacy,Fountain Pen Hospital,Frederick Douglass Pharmacy,Friendly Pharmacy,GVS @ Harlem Hospital Center,Galaxy Pharmacy,Gemini Hospitality,General Hospital,Goldberg's Pharmacy,Goldberger's Pharmacy,Golden Age Hospitality,Good Neighbor Pharmacy,Grace Gaelic Hospitality,Gracie Square Hospital,Gramercy Park Animal Hospital,Grand Health Pharmacy,Grand Pharmacy,Greentree Pharmacy,Greenwich Village Animal Hospital,Grove Drugs,HAND Hospitality,HHC Health and Hospitals,HSS,HSS - Belaire Building,HSS Rehabilitation 75th Street,Habitat Pharmacy,Habitat Specialty Pharmacy,Halpern Pharmacy,Hamilton Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospita

In [25]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,103 Pharmacy,110 pharmacy,133rd Street Pharmacy,139 Pharmacy,145th Street Animal Hospital,181 Pharmacy,678 Pharmacy,79th Street Pharmacy,8A Neuroscience Critical Care - Mt. Sinai Roosevelt Hospital,8B Neuroscience Patient Unit - Mt. Sinai Roosevelt Hospital,90th Street Pharmacy,98K Hospitality,A.M. Pharmacy II,ABC Animal Hospital,ABC Pharmacy,ABC Pharmacy / AID Pharmacy,ABC Pharmacy ; First Pharmaceutical Corp.,ABC Pharmacy V Corp,AIM Hospitality Group,APKG Pharmacy,ASPCA Adoption Center,ASPCA Animal Hospital,Abacus Pharmacy Inc.,Abc Pharmacy,Abu Dhabi Hospitality,Aid Pharmacy,Alisons Pharmacy,All Health Pharmacy,Allen Hospital Breastfeeding Fair,Alrite Pharmacy Inc.,Ambulatory Surgery at Roosevelt Hospital,American Academy of Hospitality Sciences,Animal Hospital Of Chelsea,Animal Hospital at Bideawee,Animal hospital,Apollo Pharmacy,Apotheco Pharmacy Manhattan - Dermatology Pharmacy,Apthorp Pharmacy,Arthur J. Antenucci Medical Research Building (The Roosevelt Hospital),Au Bon Pain,Avenue A Pharmacy,Avenue C pharmacy,B&B Hospitality Offices,BR Guest Hospitality,Bac Ai,Battery Park Pharmacy,Battery Park Veterinary Hospital,Bayard L.C. Pharmacy,Bayard L.C. Pharmacy Corp,Bayard Pharmacy,Bellevue Hospital 16W,Bellevue Hospital Center Gift Shop,Bellevue Hospital Center Pediatric Emergency,Bellevue Hospital Center Speciality Surgery,"Bellevue Hospital Center, 2nd Floor",Bellevue Pharmacy,Best Care Pharmacy,Best Western Plus Hospitality House,Beth Israel Hospital Department Of Neurology,Beth Israel Hospital Smoke Spot,Bickson Hospitality Group,Bio Care Pharmacy,Blake pharmacy,Bliss Pharmacy,Bliss Pharmacy Inc. / Upper Madison Drugs (DBA),Bowen Pharmacy,Bowery Pharmacy Inc,Brisely's Pharmacy,Broadway Family Pharmacy,Broadway Pharmacy,Brooklyn Navy Yard Hospital,Buy Rite Pharmacy,CCU @ Lenox Hill Hospital,CO Bigelow Pharmacy,CVS Pharmacy,CVS pharmacy,CVS/pharmacy Specialty Services,Cafe 58 @ Roosevelt Hospital,Caligore Pharmacy,Canal Street Pharmacy Inc.,Canalberry Pharmacy,CarePlus CVS/pharmacy,Carnegie Hill Pharmacy,Cathedral Dog & Cat Veterinary Hospital,Center Care Pharmacy,Century Pharmacy,Century pharmacy,Charlton Pharmacy,Chelsea Animal Hospital,Chelsea Royal Care Pharmacy,Cherry's Pharmacy,Christopher Park Veterinary Hospital,Cici Pharmacy,City Drug & Surgical Pharmacy,City Drug Pharmacy,Clean Care Pharmacy,Columbus Avenue Pharmacy,Community Pharmacy,Confucius Mannings Pharmacy,Confucius Pharmacy,Cooper Square Veterinary Hospital,Cordette Pharmacy,Cross County Pharmacy,Crown Hospitality Group,Cvs Pharmacy,Cvs Pharmacy In Target,Daliza Pharmacy,David’s Pharmacy,Denihan Hospitality Group,Department of Surgery-Bellevue Hospital,Destinations Link Hospitality,Dette Pharmacy,Dichter Pharmacy,Downtown Pharmacy,Drug Loft Pharmacy,Duane Reade,Duane Reade Pharmacy,E Care Pharmacy,ESquared Hospitality,EZ Care Pharmacy,East Village Prescription Center,Eastside Animal Hospital,Elliott Pharmacy,EmblemHealth Pharmacy,Empire Pharmacy,Erno Laszlo Counter at Apthorp Pharmacy,Esquire Pharmacy,Eve Pharmacy & Optical,"Fadi F. Attiyeh, MD",Falk Pharmacy,Family Choice Pharmacy Corp.,Family Plus Pharmacy Corp,Fancy Pharmacy,Feins Ethical Pharmacy,First Ave Veterinary Hospital and Dental Clinic,First Avenue Pharmacy,Forever Health Pharmacy,Fountain Pen Hospital,Frederick Douglass Pharmacy,Friendly Pharmacy,GVS @ Harlem Hospital Center,Galaxy Pharmacy,Gemini Hospitality,General Hospital,Goldberg's Pharmacy,Goldberger's Pharmacy,Golden Age Hospitality,Good Neighbor Pharmacy,Grace Gaelic Hospitality,Gracie Square Hospital,Gramercy Park Animal Hospital,Grand Health Pharmacy,Grand Pharmacy,Greentree Pharmacy,Greenwich Village Animal Hospital,Grove Drugs,HAND Hospitality,HHC Health and Hospitals,HSS,HSS - Belaire Building,HSS Rehabilitation 75th Street,Habitat Pharmacy,Habitat Specialty Pharmacy,Halpern Pharmacy,Hamilton Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospita

In [18]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                                             venue  freq
0                            Battery Park Pharmacy   0.5
1                 Battery Park Veterinary Hospital   0.5
2                                     103 Pharmacy   0.0
3                   New York Presbyterian Hospital   0.0
4  NewYork-Presbyterian Hospital- NeuroScience ICU   0.0


----Carnegie Hill----
                              venue  freq
0                      CVS pharmacy  0.25
1  Mount Sinai Hospital, Nephrology  0.06
2            Carnegie Hill Pharmacy  0.06
3                 Quick RX Pharmacy  0.06
4                   NuCare Pharmacy  0.06


----Central Harlem----
                       venue  freq
0           Terrace pharmacy  0.04
1          Reliance Pharmacy  0.04
2  Harlem Community Pharmacy  0.04
3            Harlem Hospital  0.04
4     Harlem Hospital Center  0.04


----Chelsea----
                              venue  freq
0                      CVS pharmacy  0.20
1  West Village Vet

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Battery Park Pharmacy,Battery Park Veterinary Hospital,radiology dept. 6 th floor NYU hospital for j...,Hartley Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospital Child Psych,Harlem Hospital Dentist
1,Carnegie Hill,CVS pharmacy,"Mount Sinai Hospital, Nephrology",EmblemHealth Pharmacy,Kings Pharmacy,NuCare Pharmacy,Drug Loft Pharmacy,Sinai 6 Childrens Hospital,Carnegie Hill Pharmacy,Quick RX Pharmacy,90th Street Pharmacy
2,Central Harlem,Terrace pharmacy,Reliance Pharmacy,Keanes Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospital Child Psych,Harlem Hospital Dentist,New York Hospitality
3,Chelsea,CVS pharmacy,Community Pharmacy,West Village Veterinary Hospital,Heart Of Chelsea Animal Hospital,New London Pharmacy,Animal Hospital Of Chelsea,Habitat Specialty Pharmacy,NYC Chelsea Pharmacy,Chelsea Royal Care Pharmacy,Chelsea Animal Hospital
4,Chinatown,Forever Health Pharmacy,Buy Rite Pharmacy,People's pharmacy,Best Care Pharmacy,Canal Street Pharmacy Inc.,Hester Pharmacy Corp,Bowery Pharmacy Inc,NOHO Pharmacy,NEW YORK PHARMACY,Bayard Pharmacy


## 4. Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Kingsbridge Pharmacy,NewYork-Presbyterian Allen Hospital Emergency ...,Friendly Pharmacy,Allen Hospital Breastfeeding Fair,NewYork-Presbyterian Och Spine Hospital,NewYork-Presbyterian/The Allen Hospital,Rite Aid,The Spine Hospital,United Pharmacy,The Allen Hospital/ Room 160
1,Manhattan,Chinatown,40.715618,-73.994279,1,Forever Health Pharmacy,Buy Rite Pharmacy,People's pharmacy,Best Care Pharmacy,Canal Street Pharmacy Inc.,Hester Pharmacy Corp,Bowery Pharmacy Inc,NOHO Pharmacy,NEW YORK PHARMACY,Bayard Pharmacy
2,Manhattan,Washington Heights,40.851903,-73.936900,1,City Drug Pharmacy,VIM Pharmacy,Twin Pharmacy,Viva Pharmacy,St. Jesus Pharmacy,Milena Pharmacy,San Pedro Pharmacy,181 Pharmacy,Hilltop Pharmacy,City Drug & Surgical Pharmacy
3,Manhattan,Inwood,40.867684,-73.921210,0,Isham Broadway Pharmacy,Sherman Pharmacy,New York Presbyterian Hospital,Dichter Pharmacy,Inwood Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Hamilton Pharmacy,Halpern Pharmacy,Harlem Hospital Center Mural Pavilion
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Sugar Hill Pharmacy,Mishkin's Pharmacy,Feins Ethical Pharmacy,Apollo Pharmacy,139 Pharmacy,145th Street Animal Hospital,Upper Manhattan Pharmacy,amsterdam pharmacy,Daliza Pharmacy,CVS pharmacy


Creating a map of clustered neighborhoods

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Isham Broadway Pharmacy,Sherman Pharmacy,New York Presbyterian Hospital,Dichter Pharmacy,Inwood Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Hamilton Pharmacy,Halpern Pharmacy,Harlem Hospital Center Mural Pavilion


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Kingsbridge Pharmacy,NewYork-Presbyterian Allen Hospital Emergency ...,Friendly Pharmacy,Allen Hospital Breastfeeding Fair,NewYork-Presbyterian Och Spine Hospital,NewYork-Presbyterian/The Allen Hospital,Rite Aid,The Spine Hospital,United Pharmacy,The Allen Hospital/ Room 160
1,Chinatown,Forever Health Pharmacy,Buy Rite Pharmacy,People's pharmacy,Best Care Pharmacy,Canal Street Pharmacy Inc.,Hester Pharmacy Corp,Bowery Pharmacy Inc,NOHO Pharmacy,NEW YORK PHARMACY,Bayard Pharmacy
2,Washington Heights,City Drug Pharmacy,VIM Pharmacy,Twin Pharmacy,Viva Pharmacy,St. Jesus Pharmacy,Milena Pharmacy,San Pedro Pharmacy,181 Pharmacy,Hilltop Pharmacy,City Drug & Surgical Pharmacy
4,Hamilton Heights,Sugar Hill Pharmacy,Mishkin's Pharmacy,Feins Ethical Pharmacy,Apollo Pharmacy,139 Pharmacy,145th Street Animal Hospital,Upper Manhattan Pharmacy,amsterdam pharmacy,Daliza Pharmacy,CVS pharmacy
5,Manhattanville,Pharmacy & Surgical Supply,Hamilton Pharmacy,Omega Pharmacy,UPS Access Point™ HERITAGE PHARMACY & SURGICAL...,Knickerbocker Hospital,Broadway Pharmacy,133rd Street Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center
6,Central Harlem,Terrace pharmacy,Reliance Pharmacy,Keanes Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospital Child Psych,Harlem Hospital Dentist,New York Hospitality
7,East Harlem,First Avenue Pharmacy,103 Pharmacy,Bliss Pharmacy,SmartRx Specialty Pharmacy,Icahn School Of Medicine Pharmacy,Maxwell Pharmacy Inc,Cross County Pharmacy,Metropolitan Pharmacy,Blake pharmacy,Marcia Pharmacy
8,Upper East Side,Lenox Hill Hospital Sleep Center,CVS pharmacy,Animal hospital,CCU @ Lenox Hill Hospital,The Pharmacy,Neurosurgery at Lenox Hill Hospital,Value Price Pharmacy,Victory Gardens Lenox Hill Hospital,Bickson Hospitality Group,Mercy Animal Hospital
9,Yorkville,CVS pharmacy,Jake's Computer Hospital,Health Wise Pharmacy,Metropolitan Hospital Labor Room,ASPCA Animal Hospital,ASPCA Adoption Center,Tisane Pharmacy & Cafe,Mark David Hospitality,NuCare Pharmacy,Mount Sinai Hospital Jackson Heights
10,Lenox Hill,CVS pharmacy,Walgreens at New York Presbyterian Hospital,Memorial Sloan Kettering Cancer Center,Cherry's Pharmacy,Goldberger's Pharmacy,NewYork-Presbyterian Hospital - Emergency Depa...,New York Presbyterian Hospital Weill Cornell M...,New York Presbyterian Hospital Weill Cornell M...,New York Presbyterian Hospital- VP Suites,HSS - Belaire Building


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Hudson Yards,Real Hospitality Regional Offices,CVS pharmacy,radiology dept. 6 th floor NYU hospital for j...,Harlem Hospital Dentist,Harlem Hospital Radiology Dept,Harlem Hospital Orthopedic Clinic2nd Fl,Harlem Hospital Kountz Pavillon,Harlem Hospital Greenmarket,Harlem Hospital Child Psych,Harmony Pharmacy


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Battery Park Pharmacy,Battery Park Veterinary Hospital,radiology dept. 6 th floor NYU hospital for j...,Hartley Pharmacy,Harlem Community Pharmacy,Harlem Hospital,Harlem Hospital Center,Harlem Hospital Center Mural Pavilion,Harlem Hospital Child Psych,Harlem Hospital Dentist
